In [1]:
import requests
from time import sleep

## Get Project ID from Project Name

In [2]:
api_key = "api-key"
url = "https://rml.azure-api.net/aws/projects/"
headers = {"Ocp-Apim-Subscription-Key" : api_key}

In [3]:
projects_list_url = url + "list"
response = requests.get(projects_list_url,headers=headers)
project_list = response.json()

In [4]:
project_list

[{'projectName': 'gateway_project',
  'projectId': '304bd50f-96f7-4980-8855-8f508ad6c083'},
 {'projectName': 'compute_project',
  'projectId': 'a9a6d345-0de8-4df3-ac72-c0b7e4742333'},
 {'projectName': 'compute_project_2',
  'projectId': 'e2ea8b60-b302-421b-9643-b6fe2e001c6b'}]

In [5]:
project_name = "compute_project"
project_id = None
for project in project_list:
    if project["projectName"] == project_name:
        project_id = project["projectId"]
        break

In [6]:
project_url = url + project_id
print(project_url)

https://rml.azure-api.net/aws/projects/a9a6d345-0de8-4df3-ac72-c0b7e4742333


## Get Project Status

In [7]:
def get_project_status(project_url):
    response = requests.get(project_url + "/status",headers=headers)
    project_status = response.json()["projectStatus"]
    return project_status

In [8]:
project_status = get_project_status(project_url)
print(project_status)

OFF


## Turn ON if the project status is OFF

In [9]:
if project_status != "OFF" or project_status != "STOPPING":
    response = requests.put(project_url + "/start",headers=headers)

In [10]:
sleep_time = 10
total_wait_time = 0
while total_wait_time < 600:
    project_status = get_project_status(project_url)
    sleep(10)
    total_wait_time += sleep_time
    print("Project Status %s"%project_status)
    if project_status == "ON":
        break

Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status STARTING
Project Status ON


## Run Commands or Shell Scripts on the compute project and command ID

In [17]:
commands = {"commands" : ["runuser -u ubuntu -- bash /home/ubuntu/efs/script.sh", "runuser -u ubuntu -- ls /home/ubuntu"]}
response = requests.post(project_url + "/execute",headers=headers,json=commands)
response.json()

{'commandId': 'cf17b952-525d-46ce-a528-bc6981362ee7', 'status': 'InProgress'}

In [18]:
command_id = response.json()["commandId"]
command_id

'cf17b952-525d-46ce-a528-bc6981362ee7'

## Get Command Status

In [19]:
response = requests.get(project_url + "/execute/" + command_id,headers=headers)
response.json()

{'output': ['/usr/local/bin:/home/ubuntu/anaconda3/bin:/home/ubuntu/.local/bin:/usr/local/bin:/home/ubuntu/anaconda3/condabin:/home/ubuntu/anaconda3/bin:/usr/local/cuda/bin:/usr/local/bin:/opt/aws/bin:/usr/local/mpi/bin:/usr/local/cuda/bin:/usr/local/bin:/opt/aws/bin:/home/ubuntu/.dl_binaries/bin:/usr/local/mpi/bin:/opt/aws/neuron/bin:/usr/local/cuda/bin:/usr/local/bin:/opt/aws/bin:/usr/local/mpi/bin:/opt/amazon/openmpi/bin/:/opt/amazon/efa/bin/:/home/ubuntu/anaconda3/bin:/usr/bin:/usr/local/bin:/opt/slurm/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin:/opt/slurm/bin/',
  'ip-10-0-0-42',
  '/home/ubuntu/anaconda3/bin/python',
  'LICENSE',
  'Nvidia_Cloud_EULA.pdf',
  'README',
  'anaconda3',
  'aws-cfn-bootstrap-2.0',
  'azhpc-templates',
  'efs',
  'examples',
  'hostfile',
  'sample.txt',
  'sample1.txt',
  'src',
  'tools',
  'tutorials'],
 'error': ['/home/ubuntu/efs/script.sh: line 6: /home/ubuntu/sample1.txt: Permission denied'],
 'status': 'Success'}

## Turn OFF if the project status is ON

In [20]:
response = requests.put(project_url + "/stop",headers=headers)
response

<Response [200]>

In [21]:
sleep_time = 10
total_wait_time = 0
while total_wait_time < 300:
    project_status = get_project_status(project_url)
    sleep(10)
    total_wait_time += sleep_time
    print("Project Status %s"%project_status)
    if project_status == "OFF":
        break

Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status STOPPING
Project Status OFF
